# =========== ANÁLISIS DE MÉTRICAS ===========

In [274]:
import pandas as pd
import numpy as np
import os

files_path = "data/"

# Eventos buscados para cada una de las metricas
eventsOfMetrics = [["GAME_START", "GAME_END", "JUMP_START", "JUMP_END"], ["GAME_START", "GAME_END", "PLAYER_MOVE"], ["GAME_START", "GAME_END", "JUMP_START", "JUMP_END"]] 

numPlatforms = 24

In [275]:
# Devuelve los datos leidos y el formato del archivo leido
def ReadFile(archivo):
    try:
        # Intenta leer el archivo como JSON
        data = pd.read_json(archivo)
        # Devuelve una lista de diccionarios
        return data.to_dict(orient='records'), 'json'
    except ValueError:
        try:
            # Intenta leer el archivo como CSV
            data = pd.read_csv(archivo)
            return data, 'csv'
        except Exception as e:
            print(f"No se pudo leer el archivo {archivo}:", e)
            return None, None

In [276]:
# Filtra los eventos que se necesitan para la metrica que se va a medir
def FilterEventsOfMetric(data, file_type, eventsOfMetric):

    if file_type == 'csv':
        if 'eventType' in data.columns:
            filtered_data = data[data['eventType'].isin(eventsOfMetric)]
            sorted_data = filtered_data.sort_values(by='timeStamp')
            return sorted_data
        else:
            print("La columna 'eventType' no está presente en el DataFrame CSV.")
            return None
        
    elif file_type == 'json':
        #Comprobar que es una lista de diccionarios
        if isinstance(data, list):      
            filtered_events = [event for event in data if event.get('eventType') in eventsOfMetric]
            if filtered_events:
                sorted_events = sorted(filtered_events, key=lambda x: x.get('timeStamp'))
                return sorted_events
            
            else:
                print("No se encontraron eventos con los tipos especificados en el diccionario JSON.")
                return None

In [277]:

def FirstMetric(events):

    failures_before_P4 = 0
    games = []

    game_start = None
    jump_start = None
    game_end = None
    jump_end = None

    for event in events:
        event_type = event.get('eventType')

        if event_type == 'GAME_START' and game_start is None:
            game_start = event

        elif event_type == 'JUMP_START':

            if game_end is not None:
                game_start = None
                game_end = None
            elif jump_start is None:
                jump_start = event

        elif event_type == 'GAME_END':

            game_end = event
            game_start = None

            # Si acaba una partida y ha habido al menos un fallo lo guardamos 
            if failures_before_P4 != 0:
                games.append(failures_before_P4)
                failures_before_P4 = 0

        elif event_type == 'JUMP_END':

            if jump_start is not None:
                jump_end = event
                platformIdJumpStart = jump_start.get('platformId')
                platformIdJumpEnd = jump_end.get('platformId')

                # Reseteamos los eventos de saltar
                jump_start = None
                jump_end = None

                # Si ya se ha llegado a la P4 ya no se cuenta más veces en esta partida
                if platformIdJumpEnd == 4:
                    game_start = None
                    game_end = None
                    continue

                # Si hay fallo 
                if platformIdJumpEnd < platformIdJumpStart:
                    failures_before_P4 += 1
    
    return games


In [278]:
def SecondMetric(events):
    
    movesInEachPlatform = np.zeros(numPlatforms, dtype=int)
    games = []

    game_start = None
    game_end = None

    print(movesInEachPlatform)

    for event in events:
        event_type = event.get('eventType')
        
        if event_type == 'GAME_START' and game_start is None:
            game_start = event

        elif event_type == 'PLAYER_MOVE':

            if game_end is not None:
                game_start = None
                game_end = None
            else:
                platformId = event.get('platformId')

                if platformId != 1:
                    print(movesInEachPlatform)
                    movesInEachPlatform[platformId-1] += 1


        elif event_type == 'GAME_END':

            game_end = event
            game_start = None

            # Si acaba una partida y ha habido al menos un fallo lo guardamos 
            if any(movesInEachPlatform):
                games.append(movesInEachPlatform)
                movesInEachPlatform = np.zeros(numPlatforms, dtype=int)
    
    return games

In [279]:
def ThirdMetric(filteredEvents):
    return 0

In [280]:
# Por cada archivo dentro de data/
for file in os.listdir(files_path):
    file_path = os.path.join(files_path, file)

    if os.path.isfile(file_path):

        data, fileType = ReadFile(file_path)
        
        if fileType == None:
            continue

        i = 0

        for eventsOfMetric in eventsOfMetrics:
            
            filteredEvents = FilterEventsOfMetric(data, fileType, eventsOfMetric)

            if i == 0:
                print(FirstMetric(filteredEvents))
            if i == 1:
                print(SecondMetric(filteredEvents))
            else:
                ThirdMetric(filteredEvents)
                
            i += 1


No se pudo leer el archivo data/eventsDani.json: Error tokenizing data. C error: Expected 2 fields in line 1216, saw 3

[5]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
def ShrekIsLoveShrekIsLive():
    print("Shrek is love")
    print("Shrek is live")


import Alvin as Calvin

while True:
    Calvin.ShrekIsLoveShrekIsLive()
